In [136]:
import pandas as pd
import numpy as np
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [112]:
premier_league_season23_24 = pd.read_csv('E0.csv')

In [114]:
teams = premier_league_season23_24['HomeTeam'].unique()

In [314]:
def clean_seasonal_data(dataset):
    dataset.drop(['HTHG','HTAG','HTR','Referee','BWH', 'BWD', 'BWA', 'IWH', 'IWD', 'IWA', 'PSH', 'PSD', 'PSA', 'WHH', 'WHD', 'WHA', 'VCH', 'VCD', 'VCA', 'MaxH', 'MaxD', 'MaxA','AvgH','AvgD',	'AvgA',	'B365>2.5',	'B365<2.5',	'P>2.5',	'P<2.5',	'Max>2.5',	'Max<2.5',	'Avg>2.5',	'Avg<2.5',	'AHh',	'B365AHH',	'B365AHA',	'PAHH',	'PAHA',	'MaxAHH',	'MaxAHA',	'AvgAHH',	'AvgAHA',	'B365CH',	'B365CD',	'B365CA',	'BWCH',	'BWCD',	'BWCA',	'IWCH',	'IWCD',	'IWCA',	'PSCH',	'PSCD',	'PSCA',	'WHCH',	'WHCD',	'WHCA',	'VCCH',	'VCCD',	'VCCA',	'MaxCH',	'MaxCD',	'MaxCA',	'AvgCH',	'AvgCD',	'AvgCA',	'B365C>2.5',	'B365C<2.5',	'PC>2.5',	'PC<2.5',	'MaxC>2.5',	'MaxC<2.5',	'AvgC>2.5',	'AvgC<2.5',	'AHCh',	'B365CAHH',	'B365CAHA',	'PCAHH',	'PCAHA',	'MaxCAHH',	'MaxCAHA',	'AvgCAHH',	'AvgCAHA' ], inplace =True, axis = 1)
    dataset = pd.get_dummies(premier_league_season23_24, columns=['HomeTeam', 'AwayTeam'])
    dummy_columns = dataset.select_dtypes(bool).columns
    dataset[dummy_columns] = dataset[dummy_columns].astype(int)
    dataset['Date'] = pd.to_datetime(dataset['Date'],format = '%d/%m/%Y' )
    return dataset


def get_team_matches(team_name, dataset):
    team_games =  dataset[dataset[dataset.filter(like=team_name).columns].gt(0.5).any(axis=1)]
    matchday_array = np.arange(1, len(team_games) + 1)
    team_games['matchday'] = matchday_array
    return team_games

def get_team_home_matches(team_name, dataset):
    team_matches = get_team_matches(team_name, dataset)
    team_home_games =  team_matches[team_matches[team_matches.filter(like=f"HomeTeam_{team_name}").columns].gt(0.5).any(axis=1)]
    return team_home_games

def get_team_away_matches(team_name, dataset):
    team_matches = get_team_matches(team_name, dataset)
    team_away_games =  team_matches[team_matches[team_matches.filter(like=f"AwayTeam_{team_name}").columns].gt(0.5).any(axis=1)]
    return team_away_games

def last_n_matches_form(team_games, team_name, n):
    team_form = [0]
    last_points =[0]
    for index, row in team_games.iterrows():
        if (row['FTR'] == 'H' and row[f"HomeTeam_{team_name}"] == 1) or (row['FTR'] == 'A' and row[f"AwayTeam_{team_name}"] == 1):
            last_points.append(3)
        elif row['FTR'] == 'D':
            last_points.append(1)
        else:
            last_points.append(0)
        if len(last_points)>n:
            del last_points[0]
        team_form.append(np.sum(last_points))
    del team_form[-1]
    return team_form


def team_current_points_total(team_name, dataset, which = 'all'):
    team_points = [0]
    last_points =[0]
    if which =='all':
        team_games =  get_team_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if (row['FTR'] == 'H' and row[f"HomeTeam_{team_name}"] == 1) or (row['FTR'] == 'A' and row[f"AwayTeam_{team_name}"] == 1):
                last_points.append(3)
            elif row['FTR'] == 'D':
                last_points.append(1)
            else:
                last_points.append(0)
            team_points.append(np.sum(last_points))     
    elif which == 'home':
        team_games =  get_team_home_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if row['FTR'] == 'H':
                last_points.append(3)
            elif row['FTR'] == 'D':
                last_points.append(1)
            else:
                last_points.append(0)
            team_points.append(np.sum(last_points))
    elif which == 'away':
        team_games =  get_team_away_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if row['FTR'] == 'A':
                last_points.append(3)
            elif row['FTR'] == 'D':
                last_points.append(1)
            else:
                last_points.append(0)
            team_points.append(np.sum(last_points))
    del team_points[-1]
    return team_points


def get_team_form(team_name, dataset, n, which = 'all'):
    if which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)   
    elif which =='all':
        team_games =  get_team_matches(team_name, dataset) 
    form_n_matches = last_n_matches_form(team_games, team_name, n)
    return form_n_matches


def get_team_goals_total(team_name, dataset, which = 'all'):
    team_goals = [0]
    goals_against = [0]
    last_goals =[0]
    last_goals_against = [0]
    team_goals_per_game = [0]
    against_goals_per_game = [0]
    if which =='all':
        team_games =  get_team_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            if row[f"HomeTeam_{team_name}"] == 1 :
                last_goals.append(row['FTHG'])
                last_goals_against.append(row['FTAG'])
            elif row[f"AwayTeam_{team_name}"] == 1:
                last_goals.append(row['FTAG'])
                last_goals_against.append(row['FTHG'])
            team_goals.append(np.sum(last_goals))
            goals_against.append(np.sum(last_goals_against))
            team_goals_per_game.append(team_goals[-1]/len(team_goals))
            against_goals_per_game.append(goals_against[-1]/len(goals_against))
    elif which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_goals.append(row['FTHG'])
            last_goals_against.append(row['FTAG'])
            team_goals.append(np.sum(last_goals))
            goals_against.append(np.sum(last_goals_against))
            team_goals_per_game.append(team_goals[-1]/len(team_goals))
            against_goals_per_game.append(goals_against[-1]/len(goals_against))
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)
        for index, row in team_games.iterrows():
            last_goals.append(row['FTAG'])
            last_goals_against.append(row['FTHG'])
            team_goals.append(np.sum(last_goals))
            goals_against.append(np.sum(last_goals_against))
            team_goals_per_game.append(team_goals[-1]/len(team_goals))
            against_goals_per_game.append(goals_against[-1]/len(goals_against))
    del team_goals[-1]
    del goals_against[-1]
    del team_goals_per_game[-1]
    del against_goals_per_game[-1]
    return team_goals, goals_against, team_goals_per_game, against_goals_per_game


def last_n_matches_goals(team_games, team_name, n):
    team_goals = [0]
    goals_against = [0]
    last_goals =[0]
    last_goals_against = [0]
    
    for index, row in team_games.iterrows():
        if row[f"HomeTeam_{team_name}"] == 1:
            last_goals.append(row['FTHG'])
            last_goals_against.append(row['FTAG'])
        elif row[f"AwayTeam_{team_name}"] == 1:
            last_goals.append(row['FTAG'])
            last_goals_against.append(row['FTHG'])
        if len(last_goals)>n:
            del last_goals[0]
            del last_goals_against[0]
        team_goals.append(np.sum(last_goals))
        goals_against.append(np.sum(last_goals_against))
    del team_goals[-1]
    del goals_against[-1]
    return team_goals, goals_against

def get_team_goals_n_matches(team_name, dataset, n, which = 'all'):
    if which =='home':
        team_games =  get_team_home_matches(team_name, dataset)
    elif which =='away':
        team_games =  get_team_away_matches(team_name, dataset)   
    elif which =='all':
        team_games =  get_team_matches(team_name, dataset) 
    team_goals_n_matches, goals_against_n_matches = last_n_matches_goals(team_games, team_name, n)
    return  team_goals_n_matches, goals_against_n_matches


In [118]:
premier_league_season23_24 = clean_seasonal_data(premier_league_season23_24)

In [330]:
team_1_games =  get_team_matches(teams[0], premier_league_season23_24)
team_1_away_games = get_team_away_matches(teams[0], premier_league_season23_24)
team_1_home_games = get_team_home_matches(teams[0], premier_league_season23_24)

In [332]:
team_1_games['total_points'] = team_current_points_total(teams[0], premier_league_season23_24)
team_1_games['5_form'] = get_team_form(teams[0], premier_league_season23_24, 5, 'all')
team_1_games['10_form'] = get_team_form(teams[0], premier_league_season23_24, 10, 'all')

team_1_all_goals_stats = get_team_goals_total(teams[0], premier_league_season23_24, 'all')
team_1_home_goals_stats = get_team_goals_total(teams[0], premier_league_season23_24, 'home')
team_1_away_goals_stats = get_team_goals_total(teams[0], premier_league_season23_24, 'away')

team_1_games['total_goals'] = team_1_all_goals_stats[0]
team_1_games['total_goals_against'] = team_1_all_goals_stats[1]
team_1_games['total_goals_per_game'] = team_1_all_goals_stats[2]
team_1_games['total_goals_against_per_game'] = team_1_all_goals_stats[3]

team_1_games['5_form_goals_scored'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 5, 'all')[0]
team_1_games['5_form_goals_against'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 5, 'all')[1]

team_1_games['10_form_goals_scored'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 10, 'all')[0]
team_1_games['10_form_goals_against'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 10, 'all')[1]

team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, 'total_points_away'] = team_current_points_total(teams[0], premier_league_season23_24, 'away')
team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, 'total_points_home'] = team_current_points_total(teams[0], premier_league_season23_24, 'home')

team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, '5_form_away'] = get_team_form(teams[0], premier_league_season23_24, 5, 'away')
team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, '10_form_away'] = get_team_form(teams[0], premier_league_season23_24, 10, 'away')

team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, '5_form_home'] = get_team_form(teams[0], premier_league_season23_24, 5, 'home')
team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, '10_form_home'] = get_team_form(teams[0], premier_league_season23_24, 10, 'home')


team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, 'total_goals_away'] = team_1_away_goals_stats[0]
team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, 'total_goals_against_away'] = team_1_away_goals_stats[1]
team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, 'total_goals_per_game_away'] = team_1_away_goals_stats[2]
team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, 'total_goals_against_per_game_away'] = team_1_away_goals_stats[3]

team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, 'total_goals_home'] = team_1_home_goals_stats[0]
team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, 'total_goals_against_home'] = team_1_home_goals_stats[1]
team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, 'total_goals_per_game_home'] = team_1_home_goals_stats[2]
team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, 'total_goals_against_per_game_home'] = team_1_home_goals_stats[3]


team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, '5_form_goals_scored_away'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 5, 'away')[0]
team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, '10_form_goals_scored_away'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 10, 'away')[0]
team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, '5_form_goals_against_away'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 5, 'away')[1]
team_1_games.loc[team_1_games[f'AwayTeam_{teams[0]}'] == 1, '10_form_goals_against_away'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 10, 'away')[1]


team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, '5_form_goals_scored_home'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 5, 'home')[0]
team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, '10_form_goals_scored_home'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 10, 'home')[0]
team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, '5_form_goals_against_home'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 5, 'home')[1]
team_1_games.loc[team_1_games[f'HomeTeam_{teams[0]}'] == 1, '10_form_goals_against_home'] = get_team_goals_n_matches(teams[0], premier_league_season23_24, 10, 'home')[1]


In [334]:
get_team_goals_n_matches(teams[0], premier_league_season23_24, 5, 'home')[0]

[0, 0, 1, 3, 3, 4, 4, 4, 7, 7, 6, 7, 8, 3, 3, 5, 5, 4, 5]

In [336]:
team_1_games.tail(12)

,Div,Date,Time,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Bournemouth,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Chelsea,HomeTeam_Crystal Palace,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Newcastle,HomeTeam_Nott'm Forest,HomeTeam_Sheffield United,HomeTeam_Tottenham,HomeTeam_West Ham,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Bournemouth,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Chelsea,AwayTeam_Crystal Palace,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Newcastle,AwayTeam_Nott'm Forest,AwayTeam_Sheffield United,AwayTeam_Tottenham,AwayTeam_West Ham,AwayTeam_Wolves,matchday,total_points,5_form,10_form,total_goals,total_goals_against,total_goals_per_game,total_goals_against_per_game,5_form_goals_scored,5_form_goals_against,10_form_goals_scored,10_form_goals_against,total_points_away,total_points_home,5_form_away,10_form_away,5_form_home,10_form_home,total_goals_away,total_goals_against_away,total_goals_per_game_away,total_goals_against_per_game_away,total_goals_home,total_goals_against_home,total_goals_per_game_home,total_goals_against_per_game_home,5_form_goals_scored_away,10_form_goals_scored_away,5_form_goals_against_away,10_form_goals_against_away,5_form_goals_scored_home,10_form_goals_scored_home,5_form_goals_against_home,10_form_goals_against_home
265,E0,2024-03-03,13:00,0,2,A,20,10,6,3,8,13,7,2,2,4,0,0,3.75,3.60,1.95,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,13,1,5,25,58,0.925926,2.148148,4,16,9,24,NaN,5.0,NaN,NaN,2.0,5.0,NaN,NaN,NaN,NaN,13.0,32.0,0.928571,2.285714,NaN,NaN,NaN,NaN,3.0,10.0,12.0,21.0
275,E0,2024-03-10,14:00,2,2,D,22,11,4,4,11,18,4,4,2,4,0,0,1.67,4.00,4.75,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,13,1,5,25,60,0.892857,2.142857,3,15,9,24,8.0,NaN,3.0,4.0,NaN,NaN,12.0,26.0,0.857143,1.857143,NaN,NaN,NaN,NaN,6.0,9.0,12.0,22.0,NaN,NaN,NaN,NaN
279,E0,2024-03-16,15:00,2,1,H,17,9,6,5,14,17,5,4,2,4,0,1,3.25,3.60,2.15,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,29,14,1,3,27,62,0.931034,2.137931,3,15,9,26,NaN,5.0,NaN,NaN,2.0,5.0,NaN,NaN,NaN,NaN,13.0,34.0,0.866667,2.266667,NaN,NaN,NaN,NaN,3.0,10.0,12.0,22.0
285,E0,2024-03-30,15:00,2,2,D,33,18,13,7,12,7,12,4,2,2,0,1,1.29,6.00,9.50,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,17,4,6,29,63,0.966667,2.100000,4,13,11,25,9.0,NaN,1.0,5.0,NaN,NaN,14.0,28.0,0.933333,1.866667,NaN,NaN,NaN,NaN,6.0,11.0,14.0,21.0,NaN,NaN,NaN,NaN
296,E0,2024-04-02,19:45,1,1,D,13,8,6,2,10,12,7,4,0,2,0,0,2.45,3.60,2.75,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,31,18,5,7,31,65,1.000000,2.096774,6,10,11,24,NaN,8.0,NaN,NaN,5.0,8.0,NaN,NaN,NaN,NaN,15.0,35.0,0.937500,2.187500,NaN,NaN,NaN,NaN,5.0,11.0,11.0,19.0
305,E0,2024-04-06,15:00,1,0,H,12,6,3,1,11,13,3,5,2,1,0,1,1.65,4.00,5.25,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,32,19,6,7,32,66,1.000000,2.062500,7,8,11,24,10.0,NaN,2.0,6.0,NaN,NaN,16.0,30.0,1.000000,1.875000,NaN,NaN,NaN,NaN,6.0,12.0,13.0,21.0,NaN,NaN,NaN,NaN
315,E0,2024-04-13,15:00,1,1,D,11,20,3,8,18,8,4,5,1,2,0,0,3.50,3.75,2.00,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,33,19,6,7,32,67,0.969697,2.030303,7,7,10,22,NaN,9.0,NaN,NaN,5.0,9.0,NaN,NaN,NaN,NaN,16.0,36.0,0.941176,2.117647,NaN,NaN,NaN,NaN,5.0,12.0,11.0,18.0
324,E0,2024-04-20,15:00,1,4,A,18,16,11,9,10,9,11,2,1,0,0,0,2.75,3.60,2.45,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,34,20,6,7,33,68,0.970588,2.000000,6,6,9,21,10.0,NaN,2.0,6.0,NaN,NaN,16.0,31.0,0.941176,1.823529,NaN,NaN,NaN,NaN,5.0,11.0,11.0,19.0,NaN,NaN,NaN,NaN
338,E0,2024-04-27,15:00,1,1,D,27,16,10,7,7,11,6,5,1,3,0

In [86]:
premier_league_season23_24

,Div,Date,Time,FTHG,FTAG,FTR,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,HomeTeam_Arsenal,HomeTeam_Aston Villa,HomeTeam_Bournemouth,HomeTeam_Brentford,HomeTeam_Brighton,HomeTeam_Burnley,HomeTeam_Chelsea,HomeTeam_Crystal Palace,HomeTeam_Everton,HomeTeam_Fulham,HomeTeam_Liverpool,HomeTeam_Luton,HomeTeam_Man City,HomeTeam_Man United,HomeTeam_Newcastle,HomeTeam_Nott'm Forest,HomeTeam_Sheffield United,HomeTeam_Tottenham,HomeTeam_West Ham,HomeTeam_Wolves,AwayTeam_Arsenal,AwayTeam_Aston Villa,AwayTeam_Bournemouth,AwayTeam_Brentford,AwayTeam_Brighton,AwayTeam_Burnley,AwayTeam_Chelsea,AwayTeam_Crystal Palace,AwayTeam_Everton,AwayTeam_Fulham,AwayTeam_Liverpool,AwayTeam_Luton,AwayTeam_Man City,AwayTeam_Man United,AwayTeam_Newcastle,AwayTeam_Nott'm Forest,AwayTeam_Sheffield United,AwayTeam_Tottenham,AwayTeam_West Ham,AwayTeam_Wolves
0,E0,2023-08-11,20:00,0,3,A,6,17,1,8,11,8,6,5,0,0,1,0,8.00,5.50,1.33,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,E0,2023-08-12,12:30,2,1,H,15,6,7,2,12,12,8,3,2,2,0,0,1.18,7.00,15.00,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,E0,2023-08-12,15:00,1,1,D,14,16,5,3,9,14,10,4,1,4,0,0,2.70,3.40,2.55,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,E0,2023-08-12,15:00,4,1,H,27,9,12,3,11,12,6,7,2,2,0,0,1.33,5.50,9.00,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,E0,2023-08-12,15:00,0,1,A,19,9,9,2,12,6,10,4,0,2,0,0,2.20,3.40,3.30,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2024-05-19,16:00,5,0,H,15,8,9,2,10,8,2,4,1,4,0,0,1.85,4.33,3.60,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
376,E0,2024-05-19,16:00,2,0,H,36,4,14,3,14,11,10,2,1,1,0,1,1.17,8.00,15.00,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
377,E0,2024-05-19,16:00,2,4,A,15,15,6,7,15,20,4,4,5,4,0,0,2.90,3.90,2.20,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
378,E0,2024-05-19,16:00,3,1,H,28,3,12,2,3,12,11,2,0,1,0,0,1.08,12.00,21.00,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


# co dodać


# wartość lineupów /suma wages zawodników?
# średnia zdobywanych/traconych bramek w sezonie(poprzednich)
# pozycja w tabeli
# podania total/dom/wyjazd średnia z ost 5/srednia sezonowa 
# posiadanie piłki dom/wyjazd
# forma dom/wyjazd -- GOTOWE
# brasmki drużyny i przeciwko -- GOTOWE
# xg

# h2h drużyny powyższe